### Web Scraper to generate hisotrical weather data

In [1]:
import requests

In [2]:
import pprint

In [3]:
from pymongo import MongoClient

In [4]:
import json

In [5]:
import time

In [6]:
from bs4 import BeautifulSoup

First, clean up my data in order to identify which dates to scrape weather on

Pulling historic weather data from Pheonix, AZ

key items:

date
<div _ngcontent-app-root-c6="" class="date ng-star-inserted">1</div>

Weather description for the day
class = <div _ngcontent-app-root-c6="" class="phrase ng-star-inserted">mostly sunny</div>

Hi
<span _ngcontent-app-root-c6="" class="hi">80°</span>

Low
<span _ngcontent-app-root-c6="" class="low">61°</span>

Rain (in inches)
<span _ngcontent-app-root-c12="" class="wu-value wu-value-to">0.21</span>

FOCUS HERE
Phoenix, AZ
https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2015-4 
- date will be the changing variable


In [25]:
r = requests.get('https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2010-1')

In [26]:
r

<Response [200]>

In [8]:
type(r)

requests.models.Response

In [27]:
r.status_code

200

In [28]:
r.content

b'<!DOCTYPE html><html lang="en" itemscope="" itemtype="http://schema.org/Organization" prefix="og: http://ogp.me/ns#"><head itemscope="" itemtype="http://schema.org/WebSite">\n  <meta charset="utf-8">\n  <title>Phoenix, AZ Weather Calendar | Weather Underground</title>\n  <base href="/">\n  <meta name="viewport" content="width=device-width, initial-scale=1">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n  <link rel="dns-prefetch" href="//widgets.outbrain.com">\n  <link rel="dns-prefetch" href="//odb.outbrain.com">\n  <link rel="dns-prefetch" href="//c.amazon-adsystem.com">\n  <link rel="dns-prefetch" href="//s.amazon-adsystem.com">\n  <link rel="dns-prefetch" href="//aax.amazon-adsystem.com">\n  <link rel="dns-prefetch" href="//partner.googleadservices.com">\n  <link rel="dns-prefetch" href="//tpc.googlesyndication.com">\n  <link rel="dns-prefetch" href="//pagead2.googlesyndication.com">\n  <link rel="dns-prefetch" href="//h.nexac.com">\n  <link rel="dns-prefetch"

In [30]:
_ = pprint.pprint(r.text)

('<!DOCTYPE html><html lang="en" itemscope="" '
 'itemtype="http://schema.org/Organization" prefix="og: '
 'http://ogp.me/ns#"><head itemscope="" itemtype="http://schema.org/WebSite">\n'
 '  <meta charset="utf-8">\n'
 '  <title>Phoenix, AZ Weather Calendar | Weather Underground</title>\n'
 '  <base href="/">\n'
 '  <meta name="viewport" content="width=device-width, initial-scale=1">\n'
 '  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n'
 '  <link rel="dns-prefetch" href="//widgets.outbrain.com">\n'
 '  <link rel="dns-prefetch" href="//odb.outbrain.com">\n'
 '  <link rel="dns-prefetch" href="//c.amazon-adsystem.com">\n'
 '  <link rel="dns-prefetch" href="//s.amazon-adsystem.com">\n'
 '  <link rel="dns-prefetch" href="//aax.amazon-adsystem.com">\n'
 '  <link rel="dns-prefetch" href="//partner.googleadservices.com">\n'
 '  <link rel="dns-prefetch" href="//tpc.googlesyndication.com">\n'
 '  <link rel="dns-prefetch" href="//pagead2.googlesyndication.com">\n'
 '  <link rel=

 '.city-star{display:block;width:44px;height:44px;padding:0;position:absolute;top:0;right:0;background-image:url(//www.wunderground.com/static/i/wu/touchStars.png);background-position:0 '
 '0}  .favorites-list ul .is-city .city-star:hover,   .favorites-mobile ul '
 '.is-city .city-star:hover,   .ui-autocomplete .is-city '
 '.city-star:hover{background-position:0 -88px}  .favorites-list ul .is-city '
 '.city-home,   .favorites-mobile ul .is-city .city-home,   .ui-autocomplete '
 '.is-city '
 '.city-home{display:block;width:26px;height:24px;padding:0;position:absolute;top:10px;right:0;background-image:url(//www.wunderground.com/static/i/wu/setHome.png);background-position:0 '
 '0}  .favorites-list ul .is-city .city-home.active,   .favorites-list ul '
 '.is-city .city-home:hover,   .favorites-mobile ul .is-city '
 '.city-home.active,   .favorites-mobile ul .is-city .city-home:hover,   '
 '.ui-autocomplete .is-city .city-home.active,   .ui-autocomplete .is-city '
 '.city-home:hover{backgro

 '.mat-raised-button[disabled][disabled]{color:rgba(0,0,0,.26);background-color:rgba(0,0,0,.12)}  '
 '.mat-fab.mat-primary .mat-ripple-element,   .mat-flat-button.mat-primary '
 '.mat-ripple-element,   .mat-mini-fab.mat-primary .mat-ripple-element,   '
 '.mat-raised-button.mat-primary '
 '.mat-ripple-element{background-color:rgba(0,0,0,.1)}  .mat-fab.mat-accent '
 '.mat-ripple-element,   .mat-flat-button.mat-accent .mat-ripple-element,   '
 '.mat-mini-fab.mat-accent .mat-ripple-element,   '
 '.mat-raised-button.mat-accent '
 '.mat-ripple-element{background-color:rgba(0,0,0,.1)}  .mat-fab.mat-warn '
 '.mat-ripple-element,   .mat-flat-button.mat-warn .mat-ripple-element,   '
 '.mat-mini-fab.mat-warn .mat-ripple-element,   .mat-raised-button.mat-warn '
 '.mat-ripple-element{background-color:rgba(255,255,255,.1)}  '
 '.mat-flat-button:not([class*=mat-elevation-z]),   '
 '.mat-stroked-button:not([class*=mat-elevation-z]){box-shadow:0 0 0 0 '
 'rgba(0,0,0,.2),0 0 0 0 rgba(0,0,0,.14),0 0 0 0 

 '.mat-toolbar{background:#f5f5f5;color:rgba(0,0,0,.87)}  '
 '.mat-toolbar.mat-primary{background:#f7f7f7;color:#000}  '
 '.mat-toolbar.mat-accent{background:#bbe4ff;color:#000}  '
 '.mat-toolbar.mat-warn{background:#d5202a;color:#fff}  .mat-toolbar '
 '.mat-focused .mat-form-field-ripple,   .mat-toolbar '
 '.mat-form-field-ripple,   .mat-toolbar '
 '.mat-form-field-underline{background-color:currentColor}  .mat-toolbar '
 '.mat-focused .mat-form-field-label,   .mat-toolbar .mat-form-field-label,   '
 '.mat-toolbar .mat-form-field.mat-focused .mat-select-arrow,   .mat-toolbar '
 '.mat-select-arrow,   .mat-toolbar .mat-select-value{color:inherit}  '
 '.mat-toolbar .mat-input-element{caret-color:currentColor}  '
 '.mat-tooltip{background:rgba(97,97,97,.9)}  '
 '.mat-tree{font-family:Roboto,"Helvetica Neue",sans-serif;background:#fff}  '
 '.mat-nested-tree-node,   '
 '.mat-tree-node{font-weight:400;font-size:14px;color:rgba(0,0,0,.87)}  '
 '.mat-snack-bar-container{color:rgba(255,255,255,

 '.mat-form-field-appearance-fill.mat-form-field-disabled '
 '.mat-form-field-flex{background-color:rgba(255,255,255,.05)}  .wu-dark-theme '
 '.mat-form-field-appearance-fill '
 '.mat-form-field-underline::before{background-color:rgba(255,255,255,.5)}  '
 '.wu-dark-theme .mat-form-field-appearance-fill.mat-form-field-disabled '
 '.mat-form-field-label{color:rgba(255,255,255,.5)}  .wu-dark-theme '
 '.mat-form-field-appearance-fill.mat-form-field-disabled '
 '.mat-form-field-underline::before{background-color:transparent}  '
 '.wu-dark-theme .mat-form-field-appearance-outline '
 '.mat-form-field-outline{color:rgba(255,255,255,.3)}  .wu-dark-theme '
 '.mat-form-field-appearance-outline '
 '.mat-form-field-outline-thick{color:#fff}  .wu-dark-theme '
 '.mat-form-field-appearance-outline.mat-focused '
 '.mat-form-field-outline-thick{color:#1e2023}  .wu-dark-theme '
 '.mat-form-field-appearance-outline.mat-focused.mat-accent '
 '.mat-form-field-outline-thick{color:#bbe4ff}  .wu-dark-theme '
 

 '.fi-social-skype:before{content:"\\f1da"}  '
 '.fi-social-smashing-mag:before{content:"\\f1db"}  '
 '.fi-social-snapchat:before{content:"\\f1dc"}  '
 '.fi-social-spotify:before{content:"\\f1dd"}  '
 '.fi-social-squidoo:before{content:"\\f1de"}  '
 '.fi-social-stack-overflow:before{content:"\\f1df"}  '
 '.fi-social-steam:before{content:"\\f1e0"}  '
 '.fi-social-stumbleupon:before{content:"\\f1e1"}  '
 '.fi-social-treehouse:before{content:"\\f1e2"}  '
 '.fi-social-tumblr:before{content:"\\f1e3"}  '
 '.fi-social-twitter:before{content:"\\f1e4"}  '
 '.fi-social-vimeo:before{content:"\\f1e5"}  '
 '.fi-social-windows:before{content:"\\f1e6"}  '
 '.fi-social-xbox:before{content:"\\f1e7"}  '
 '.fi-social-yahoo:before{content:"\\f1e8"}  '
 '.fi-social-yelp:before{content:"\\f1e9"}  '
 '.fi-social-youtube:before{content:"\\f1ea"}  '
 '.fi-social-zerply:before{content:"\\f1eb"}  '
 '.fi-social-zurb:before{content:"\\f1ec"}  '
 '.fi-sound:before{content:"\\f1ed"}  .fi-star:before{content:"\\f1ee

 '(min-width:40em) and (max-width:63.9375em){  '
 '.region-content-sidebar-1{margin-top:10px}  '
 '.region-content-details-sidebar{position:absolute;right:10px;left:50%}}  '
 '.wu-paid .mid-leader{display:none}  '
 '.mat-drawer-container.mat-sidenav-container{overflow:unset}  '
 '.mat-drawer-content.mat-sidenav-content{overflow:unset}[_nghost-sc2]     '
 '.ad-wrap{margin-bottom:20px}[_nghost-sc2]     '
 '.ad-wrap.top-banner{margin:0;padding-bottom:10px}.section[_ngcontent-sc2]{position:relative}</style><style '
 'ng-transition="app-root">@charset "UTF-8";.city-header[_ngcontent-sc19]   '
 'h1[_ngcontent-sc19]{margin-bottom:0;font-size:1.5rem;vertical-align:top}.city-header[_ngcontent-sc19]   '
 'h1[_ngcontent-sc19]   '
 '.material-icons[_ngcontent-sc19]{font-size:28px;width:32px}.city-header[_ngcontent-sc19]   '
 '.icons[_ngcontent-sc19]{overflow:hidden;position:absolute;padding:3px 0 0 '
 '5px;white-space:nowrap}.city-header[_ngcontent-sc19]   '
 '.station-condition[_ngcontent-sc19]{m

 'ng-transition="app-root">.mat-menu-panel{min-width:112px;max-width:280px;overflow:auto;-webkit-overflow-scrolling:touch;max-height:calc(100vh '
 '- '
 '48px);border-radius:4px;outline:0;min-height:64px}.mat-menu-panel.ng-animating{pointer-events:none}@media '
 '(-ms-high-contrast:active){.mat-menu-panel{outline:solid '
 '1px}}.mat-menu-content:not(:empty){padding-top:8px;padding-bottom:8px}.mat-menu-item{-webkit-user-select:none;-moz-user-select:none;-ms-user-select:none;user-select:none;cursor:pointer;outline:0;border:none;-webkit-tap-highlight-color:transparent;white-space:nowrap;overflow:hidden;text-overflow:ellipsis;display:block;line-height:48px;height:48px;padding:0 '
 '16px;text-align:left;text-decoration:none;max-width:100%;position:relative}.mat-menu-item::-moz-focus-inner{border:0}.mat-menu-item[disabled]{cursor:default}[dir=rtl] '
 '.mat-menu-item{text-align:right}.mat-menu-item '
 '.mat-icon{margin-right:16px;vertical-align:middle}.mat-menu-item .mat-icon '
 'svg{vertical

 'class="desktop">Thursday</div><div _ngcontent-sc112="" '
 'class="mobile">Thu</div></li><li _ngcontent-sc112="" class="calendar-weekday '
 'ng-star-inserted"><div _ngcontent-sc112="" class="desktop">Friday</div><div '
 '_ngcontent-sc112="" class="mobile">Fri</div></li><li _ngcontent-sc112="" '
 'class="calendar-weekday ng-star-inserted"><div _ngcontent-sc112="" '
 'class="desktop">Saturday</div><div _ngcontent-sc112="" '
 'class="mobile">Sat</div></li></ul><ul _ngcontent-sc112="" '
 'class="calendar-days"><!----></ul></div></lib-city-calendar></div></div><!----><div '
 '_ngcontent-sc111="" class="row is-sidebar ng-star-inserted"><div '
 '_ngcontent-sc111="" class="region-sidebar"><ad-wx-top-300-var '
 '_ngcontent-sc111="" libsticky=""><div class="atf-box ad-wrap ad-desktop" '
 'id="atf-box-browser"><div '
 'id="WX_Top300Variable"></div></div></ad-wx-top-300-var><ad-mw-position-2 '
 '_ngcontent-sc111="" _nghost-sc21=""><div _ngcontent-sc21="" class="ad-wrap '
 'ad-mobile" id="position

In [20]:
client = MongoClient('localhost', 27017)
db = client.weather
pages = db.main_pages

In [21]:
client.server_info()

{'version': '4.2.3',
 'gitVersion': '6874650b362138df74be53d366bbefc321ea32d4',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [4, 2, 3, 0],
 'openssl': {'running': 'OpenSSL 1.1.1  11 Sep 2018',
  'compiled': 'OpenSSL 1.1.1  11 Sep 2018'},
 'buildEnvironment': {'distmod': 'ubuntu1804',
  'distarch': 'x86_64',
  'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.2.0',
  'ccflags': '-fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -fno-builtin-memcmp',
  'cxx': '/opt/mongodbtoolchain/v3/bin/g++: g++ (GCC) 8.2.0',
  'cxxflags': '-Woverloaded-virtual -Wno-maybe-uninitialized -fsized-deallocation -std=c++17',
  'linkflags': '-pthread -Wl,-z,now -rdynamic -Wl,--fatal-war

In [14]:
# Seems like I'll need to set up a web scraper for each year and have the month values change

url_lst = []

def web_scraper_links():
    url = 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2015-1'
    for x in range(1,13):
        for y in range(0,10):
            url_lst.append(f"https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/201{y}-{x}")
        
        
web_scraper_links()

In [15]:
url_lst

['https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2010-1',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2011-1',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2012-1',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2013-1',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2014-1',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2015-1',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2016-1',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2017-1',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2018-1',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-1',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2010-2',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2011-2',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2012-2',
 'https://www.wunderground.com/calendar/us/az/phoen

In [16]:
len(url_lst)

120

In [17]:
def link_check():
    for x in url_lst:
        if x == "https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2011-8":
            return "that shtuff here"

In [18]:
link_check()

'that shtuff here'

In [22]:
def web_scraper(lst):
    for x in lst:
        r = requests.get(f"{x}")
        pages.insert_one({'html': r.content})

In [24]:
# web_scraper(url_lst)

KeyboardInterrupt: 